In [1]:
from Gaugi import load as gaugi_load
from kepler.pandas.readers import load as kepler_load

import pandas as pd
import numpy as np
import os

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
def load_as_df(filepath: str) -> pd.DataFrame:
    data = dict(np.load(filepath))
    df_data = pd.DataFrame(data['data'], columns=data['features'])
    df_target = pd.DataFrame(data['target'], columns=['target'])
    df = pd.concat([df_data, df_target], axis=1)
    return df

In [3]:
et=4
eta=0
dataset = 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97'
loose_dataset = 'data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
homepath = os.path.expanduser('~')
datapath = os.path.join(homepath, 'data', dataset)
loose_datapath = os.path.join(homepath, 'data', loose_dataset)
filepath = os.path.join(datapath, dataset + '_et{et}_eta{eta}.npz')
refpath = os.path.join(datapath, 'references')
reffilepath = os.path.join(refpath, dataset + '_et{et}_eta{eta}.ref.pic.gz')
loose_filepath = os.path.join(loose_datapath, loose_dataset + '_et{et}_eta{eta}.npz')

In [4]:
trig_tight='T0HLTElectronT2CaloTight'
trig_medium = 'T0HLTElectronT2CaloMedium'
trig_loose = 'T0HLTElectronT2CaloLoose'
trig_vloose = 'T0HLTElectronT2CaloVLoose'
l2calo_trigers = ['T0HLTElectronT2CaloTight', 'T0HLTElectronT2CaloMedium', 'T0HLTElectronT2CaloLoose', 'T0HLTElectronT2CaloVLoose']

In [5]:
ref = gaugi_load(reffilepath.format(et=et, eta=eta))
ref_total = ref['bkgRef']['medium_cutbased']['total'] + ref['sgnRef']['medium_cutbased']['total']
ref

{'class': 'Reference_v1',
 '__module': 'saphyra.readers.versions.Reference_v1',
 'etBinIdx': array(4),
 'etBins': array([1.5e+01, 2.0e+01, 3.0e+01, 4.0e+01, 5.0e+01, 1.0e+06]),
 '__version': 1,
 'bkgRef': OrderedDict([('tight_cutbased',
               {'total': 3769656,
                'passed': 763817.0,
                'reference': 'T0HLTElectronT2CaloTight'}),
              ('medium_cutbased',
               {'total': 3769656,
                'passed': 925500.0,
                'reference': 'T0HLTElectronT2CaloMedium'}),
              ('loose_cutbased',
               {'total': 3769656,
                'passed': 3444525.0,
                'reference': 'T0HLTElectronT2CaloLoose'}),
              ('vloose_cutbased',
               {'total': 3769656,
                'passed': 3481839.0,
                'reference': 'T0HLTElectronT2CaloVLoose'})]),
 'etaBinIdx': array(4),
 'etaBins': array([0.  , 0.8 , 1.37, 1.54, 2.37, 2.5 ]),
 'sgnRef': OrderedDict([('tight_cutbased',
               {

In [6]:
type(ref)

dict

In [6]:
medium_data = load_as_df(filepath.format(et=et, eta=eta))
medium_total = medium_data.shape[0]
print(medium_data.shape)
print(medium_data['target'].dtype)
print(medium_data['el_lhtight'].dtype)
medium_data['el_lhtight'] = medium_data['el_lhtight'].astype('int16')
print(medium_data['el_lhmedium'].dtype)
medium_data['el_lhmedium'] = medium_data['el_lhmedium'].astype('int16')
print(medium_data['el_lhloose'].dtype)
medium_data['el_lhloose'] = medium_data['el_lhloose'].astype('int16')
print(medium_data['el_lhvloose'].dtype)
medium_data['el_lhvloose'] = medium_data['el_lhvloose'].astype('int16')
print(medium_data[trig_tight].dtype)
medium_data[trig_tight] = medium_data[trig_tight].astype('int16')
print(medium_data[trig_medium].dtype)
medium_data[trig_medium] = medium_data[trig_medium].astype('int16')
print(medium_data[trig_loose].dtype)
medium_data[trig_loose] = medium_data[trig_loose].astype('int16')
print(medium_data[trig_vloose].dtype)
medium_data[trig_vloose] = medium_data[trig_vloose].astype('int16')

(5469197, 155)
int16
float32
float32
float32
float32
float32
float32
float32
float32


In [7]:
((medium_data['target'] == medium_data['el_lhtight']).all(), 
(medium_data['target'] == medium_data['el_lhmedium']).all(), 
(medium_data['target'] == medium_data['el_lhloose']).all(), 
(medium_data['target'] == medium_data['el_lhvloose']).all())

(True, True, False, False)

In [8]:
((medium_data['target'] == medium_data[trig_tight]).all(), 
 (medium_data['target'] == medium_data[trig_medium]).all(), 
 (medium_data['target'] == medium_data[trig_loose]).all(), 
 (medium_data['target'] == medium_data[trig_vloose]).all())

(False, False, False, False)

In [24]:
(medium_data['target'] == 1).sum()/(medium_data[trig_medium] == 1).sum()

0.6485837211893811

In [9]:
filtered_medium =  medium_data.loc[((medium_data['target'] == 1)&(medium_data[trig_medium] == 1)) | ((medium_data['target'] != 1)&(medium_data[trig_vloose] !=1))]
filtered_medium_total = filtered_medium.shape[0]
filtered_medium.shape

(1982705, 155)

In [ ]:
filtered_medium['target']==1).sum()

In [10]:
((filtered_medium['target'] == filtered_medium['el_lhtight']).all(), 
(filtered_medium['target'] == filtered_medium['el_lhmedium']).all(), 
(filtered_medium['target'] == filtered_medium['el_lhloose']).all(), 
(filtered_medium['target'] == filtered_medium['el_lhvloose']).all())

(True, True, False, False)

In [11]:
((filtered_medium['target'] == filtered_medium[trig_tight]).all(), 
 (filtered_medium['target'] == filtered_medium[trig_medium]).all(), 
 (filtered_medium['target'] == filtered_medium[trig_loose]).all(), 
 (filtered_medium['target'] == filtered_medium[trig_vloose]).all())

(False, True, True, True)

In [12]:
for col in medium_data.columns:
    if not col.startswith('L2Calo_ring'):
        print(col)

avgmu
L2Calo_et
L2Calo_eta
L2Calo_phi
L2Calo_reta
L2Calo_ehad1
L2Calo_eratio
L2Calo_f1
L2Calo_f3
L2Calo_weta2
L2Calo_wstot
L2Calo_e2tsts1
L2Electron_hastrack
L2Electron_pt
L2Electron_eta
L2Electron_phi
L2Electron_caloEta
L2Electron_trkClusDeta
L2Electron_trkClusDphi
L2Electron_etOverPt
et
eta
phi
rhad1
rhad
f3
weta2
rphi
reta
wtots1
eratio
f1
hastrack
numberOfBLayerHits
numberOfPixelHits
numberOfTRTHits
d0
d0significance
eProbabilityHT
trans_TRT_PID
deltaEta1
deltaPhi2
deltaPhi2Rescaled
DeltaPOverP
deltaR
eeMass
el_lhtight
el_lhmedium
el_lhloose
el_lhvloose
T0HLTElectronT2CaloTight
T0HLTElectronT2CaloMedium
T0HLTElectronT2CaloLoose
T0HLTElectronT2CaloVLoose
target


In [13]:
vloose_data = kepler_load(loose_filepath.format(et=et, eta=eta))
vloose_total = vloose_data.shape[0]
vloose_data.shape

(5599840, 216)

In [14]:
for col in vloose_data.columns:
    if not col.startswith('trig_L2_cl_ring'):
        print(col)

RunNumber
avgmu
trig_L2_cl_et
trig_L2_cl_eta
trig_L2_cl_phi
trig_L2_cl_reta
trig_L2_cl_ehad1
trig_L2_cl_eratio
trig_L2_cl_f1
trig_L2_cl_f3
trig_L2_cl_weta2
trig_L2_cl_wstot
trig_L2_cl_e2tsts1
trig_L2_el_hastrack
trig_L2_el_pt
trig_L2_el_eta
trig_L2_el_phi
trig_L2_el_caloEta
trig_L2_el_trkClusDeta
trig_L2_el_trkClusDphi
trig_L2_el_etOverPt
trig_EF_cl_hascluster
trig_EF_cl_et
trig_EF_cl_eta
trig_EF_cl_etaBE2
trig_EF_cl_phi
trig_EF_el_hascand
trig_EF_el_et
trig_EF_el_eta
trig_EF_el_etaBE2
trig_EF_el_phi
trig_EF_el_rhad1
trig_EF_el_rhad
trig_EF_el_f3
trig_EF_el_weta2
trig_EF_el_rphi
trig_EF_el_reta
trig_EF_el_wtots1
trig_EF_el_eratio
trig_EF_el_f1
trig_EF_el_hastrack
trig_EF_el_deltaEta1
trig_EF_el_deltaPhi2
trig_EF_el_deltaPhi2Rescaled
trig_EF_el_lhtight
trig_EF_el_lhmedium
trig_EF_el_lhloose
trig_EF_el_lhvloose
el_et
el_eta
el_etaBE2
el_phi
el_rhad1
el_rhad
el_f3
el_weta2
el_rphi
el_reta
el_wtots1
el_eratio
el_f1
el_hastrack
el_numberOfBLayerHits
el_numberOfPixelHits
el_numberOfTRTHits
e

In [15]:
((vloose_data['target'] == vloose_data['el_lhtight']).all(), 
(vloose_data['target'] == vloose_data['el_lhmedium']).all(), 
(vloose_data['target'] == vloose_data['el_lhloose']).all(), 
(vloose_data['target'] == vloose_data['el_lhvloose']).all())

(False, False, False, True)

In [16]:
filtered_vloose = vloose_data.loc[((vloose_data.target == 1)&(vloose_data.el_lhmedium == 1)) | ((vloose_data.target != 1)&(vloose_data.el_lhvloose !=1))]
filtered_vloose_total = filtered_vloose.shape[0]
filtered_vloose.shape

(5469055, 216)

In [18]:
((filtered_vloose['target'] == filtered_vloose['el_lhtight']).all(), 
(filtered_vloose['target'] == filtered_vloose['el_lhmedium']).all(), 
(filtered_vloose['target'] == filtered_vloose['el_lhloose']).all(), 
(filtered_vloose['target'] == filtered_vloose['el_lhvloose']).all())

(False, True, True, True)

In [17]:
ref_total, medium_total, filtered_medium_total, vloose_total, filtered_vloose_total

(5420314, 5469197, 1982705, 5599840, 5469055)

In [21]:
ref['sgnRef']['medium_cutbased']

{'total': 1650658,
 'passed': 1646125.0,
 'reference': 'T0HLTElectronT2CaloMedium'}